# 🧹 K-Style 이커머스 고객 지원 에이전트 - 리소스 정리

이 노트북은 **이커머스 프로젝트에서만 생성한** 모든 AWS 리소스를 안전하게 정리하는 프로세스를 제공합니다.

## 개요

이 정리 프로세스는 **오직 이커머스 프로젝트 관련 리소스만** 제거합니다:
- **메모리**: `/app/ecommerce/` 경로의 AgentCore Memory 리소스
- **런타임**: 이커머스 프로젝트의 Agent 런타임 인스턴스
- **게이트웨이**: 이커머스 프로젝트의 AgentCore Gateway 및 Lambda 함수들
- **보안**: `EcommerceAgentCore*` 패턴의 IAM 역할 및 `ecommerce_customer_support_agent` 시크릿
- **관측성**: 이커머스 프로젝트 관련 CloudWatch 로그
- **CloudFormation**: `ecommerce-customer-support-*` 스택들

⚠️ **안전성**: 다른 프로젝트의 리소스는 건드리지 않습니다. 이커머스 프로젝트에서 생성한 리소스만 정확히 식별하여 삭제합니다.

---

## Step 1: 필수 의존성 가져오기

정리 프로세스에 필요한 모든 모듈과 헬퍼 함수들을 로드합니다.

In [ ]:
import boto3
import os
import json
import time
from botocore.exceptions import ClientError

# AgentCore 관련 import
from bedrock_agentcore_starter_toolkit import Runtime

# Lab 헬퍼 모듈들
import sys; sys.path.insert(0, '..'); from src.helpers.utils import get_ssm_parameter

# AWS 클라이언트 설정
REGION = "us-east-1"
session = boto3.Session(region_name=REGION)
ssm_client = session.client('ssm')
secrets_client = session.client('secretsmanager')
cloudformation_client = session.client('cloudformation')
logs_client = session.client('logs')
cognito_client = session.client('cognito-idp')
iam_client = session.client('iam')
lambda_client = session.client('lambda')

# Gateway 클라이언트 별도 설정 (bedrock-agentcore-control 사용)
try:
    bedrock_agentcore_control_client = session.client('bedrock-agentcore-control')
    print("✅ bedrock-agentcore-control 클라이언트 설정 완료")
except Exception as e:
    print(f"⚠️ bedrock-agentcore-control 클라이언트 설정 실패: {e}")
    bedrock_agentcore_control_client = None

print("✅ 의존성 가져오기 완료")
print(f"🌍 작업 리전: {REGION}")
print("🎯 이커머스 프로젝트 리소스만 안전하게 정리합니다")

## Step 2: 이커머스 메모리 리소스 정리

`/app/ecommerce/` 경로의 AgentCore Memory 리소스만 제거합니다.

In [ ]:
print("🧠 이커머스 메모리 정리 시작...")

try:
    # 이커머스 프로젝트의 특정 SSM 파라미터에서만 메모리 ID 가져오기
    try:
        memory_id = get_ssm_parameter("/app/ecommerce/agentcore/memory_id")
        print(f"  📋 이커머스 메모리 ID 발견: {memory_id}")
        
        # 이커머스 메모리 리소스만 삭제
        from bedrock_agentcore.memory import MemoryClient
        memory_client = MemoryClient()
        
        # 메모리 데이터 삭제 (이커머스 프로젝트에서 생성한 것만)
        print("  🗑️ 이커머스 메모리 데이터 삭제 중...")
        memory_client.delete(memory_id)
        print("  ✅ 이커머스 메모리 데이터 삭제 완료")
        
        # 이커머스 프로젝트의 SSM 파라미터만 삭제
        ssm_client.delete_parameter(Name="/app/ecommerce/agentcore/memory_id")
        print("  ✅ 이커머스 메모리 SSM 파라미터 삭제 완료")
        
    except ssm_client.exceptions.ParameterNotFound:
        print("  ⚠️ 이커머스 메모리 ID를 찾을 수 없습니다 (이미 삭제되었거나 생성되지 않음)")
    except Exception as e:
        print(f"  ❌ 이커머스 메모리 정리 중 오류: {e}")
        
except Exception as e:
    print(f"  ❌ 메모리 정리 중 전체 오류: {e}")

print("✅ 이커머스 메모리 정리 완료")

## Step 3: 이커머스 런타임 리소스 정리

이커머스 프로젝트의 AgentCore Runtime만 제거합니다.

In [ ]:
print("🚀 이커머스 런타임 정리 시작...")

try:
    # 이커머스 프로젝트의 특정 SSM 파라미터에서만 런타임 ARN 가져오기
    try:
        runtime_arn = get_ssm_parameter("/app/ecommerce/agentcore/runtime_arn")
        print(f"  📋 이커머스 런타임 ARN 발견: {runtime_arn}")
        
        # 이커머스 런타임 인스턴스만 삭제
        runtime = Runtime()
        print("  🗑️ 이커머스 런타임 인스턴스 삭제 중...")
        runtime.delete()
        print("  ✅ 이커머스 런타임 인스턴스 삭제 완료")
        
        # 이커머스 프로젝트의 SSM 파라미터만 삭제
        ssm_client.delete_parameter(Name="/app/ecommerce/agentcore/runtime_arn")
        print("  ✅ 이커머스 런타임 SSM 파라미터 삭제 완료")
        
    except ssm_client.exceptions.ParameterNotFound:
        print("  ⚠️ 이커머스 런타임 ARN을 찾을 수 없습니다 (이미 삭제되었거나 생성되지 않음)")
    except Exception as e:
        print(f"  ❌ 이커머스 런타임 정리 중 오류: {e}")
        
except Exception as e:
    print(f"  ❌ 런타임 정리 중 전체 오류: {e}")

print("✅ 이커머스 런타임 정리 완료")

## Step 4: 이커머스 게이트웨이 리소스 정리

이커머스 프로젝트의 AgentCore Gateway와 관련 Lambda 함수들만 제거합니다.

In [ ]:
print("⚙️ 이커머스 게이트웨이 정리 시작...")

try:
    # 이커머스 프로젝트의 특정 SSM 파라미터에서만 게이트웨이 ID 가져오기
    try:
        gateway_id = get_ssm_parameter("/app/ecommerce/agentcore/gateway_id")
        print(f"  📋 이커머스 게이트웨이 ID 발견: {gateway_id}")
        
        # bedrock-agentcore-control 클라이언트로 게이트웨이 관리
        if bedrock_agentcore_control_client:
            # 먼저 이커머스 게이트웨이의 모든 타겟들 삭제
            try:
                print("  🗑️ 이커머스 게이트웨이 타겟들 삭제 중...")
                targets_response = bedrock_agentcore_control_client.list_gateway_targets(
                    gatewayIdentifier=gateway_id
                )
                
                for target in targets_response.get('targets', []):
                    target_id = target['targetId']
                    print(f"    🎯 이커머스 타겟 삭제: {target_id}")
                    bedrock_agentcore_control_client.delete_gateway_target(
                        gatewayIdentifier=gateway_id,
                        targetIdentifier=target_id
                    )
                print("  ✅ 이커머스 게이트웨이 타겟들 삭제 완료")
            except Exception as e:
                print(f"  ⚠️ 이커머스 타겟 삭제 중 오류: {e}")
            
            # 이커머스 게이트웨이만 삭제
            try:
                print("  🗑️ 이커머스 게이트웨이 삭제 중...")
                bedrock_agentcore_control_client.delete_gateway(
                    gatewayIdentifier=gateway_id
                )
                print("  ✅ 이커머스 게이트웨이 삭제 완료")
            except Exception as e:
                print(f"  ❌ 이커머스 게이트웨이 삭제 실패: {e}")
        else:
            print("  ⚠️ bedrock-agentcore-control 클라이언트를 사용할 수 없어 게이트웨이 직접 삭제를 건너뜁니다")
        
        # 이커머스 프로젝트의 SSM 파라미터만 삭제
        ssm_client.delete_parameter(Name="/app/ecommerce/agentcore/gateway_id")
        print("  ✅ 이커머스 게이트웨이 SSM 파라미터 삭제 완료")
        
    except ssm_client.exceptions.ParameterNotFound:
        print("  ⚠️ 이커머스 게이트웨이 ID를 찾을 수 없습니다 (이미 삭제되었거나 생성되지 않음)")
    except Exception as e:
        print(f"  ❌ 이커머스 게이트웨이 정리 중 오류: {e}")
        
    # 이커머스 프로젝트의 CloudFormation 스택만 삭제
    try:
        print("  🗑️ 이커머스 CloudFormation 스택 삭제 중...")
        ecommerce_stacks_to_delete = [
            "ecommerce-customer-support-infrastructure",
            "ecommerce-customer-support-cognito"
        ]
        
        for stack_name in ecommerce_stacks_to_delete:
            try:
                # 스택이 존재하는지 확인
                stacks = cloudformation_client.describe_stacks(StackName=stack_name)
                cloudformation_client.delete_stack(StackName=stack_name)
                print(f"    ✅ 이커머스 스택 삭제 요청: {stack_name}")
            except cloudformation_client.exceptions.ClientError as e:
                if "does not exist" in str(e):
                    print(f"    ⚠️ 이커머스 스택을 찾을 수 없습니다: {stack_name}")
                else:
                    print(f"    ❌ 이커머스 스택 {stack_name} 삭제 실패: {e}")
                    
    except Exception as e:
        print(f"  ⚠️ 이커머스 CloudFormation 스택 삭제 중 오류: {e}")
        
except Exception as e:
    print(f"  ❌ 게이트웨이 정리 중 전체 오류: {e}")

print("✅ 이커머스 게이트웨이 정리 완료")

## Step 5: 이커머스 보안 리소스 정리

이커머스 프로젝트에서만 생성한 IAM 역할, Cognito 사용자 풀, Secrets Manager 시크릿만 제거합니다.

In [ ]:
print("🛡️ 이커머스 보안 리소스 정리 시작...")

try:
    # 이커머스 프로젝트의 Secrets Manager 시크릿만 가져오기 및 삭제
    try:
        print("  🗑️ 이커머스 Secrets Manager 시크릿 삭제 중...")
        secret_response = secrets_client.get_secret_value(SecretId="ecommerce_customer_support_agent")
        cognito_config = json.loads(secret_response['SecretString'])
        
        # 이커머스 프로젝트의 Cognito 사용자 풀만 삭제
        try:
            print("  🗑️ 이커머스 Cognito 사용자 풀 삭제 중...")
            pool_id = cognito_config.get('pool_id')
            if pool_id:
                cognito_client.delete_user_pool(UserPoolId=pool_id)
                print(f"    ✅ 이커머스 사용자 풀 삭제 완료: {pool_id}")
        except Exception as e:
            print(f"    ⚠️ 이커머스 Cognito 사용자 풀 삭제 실패: {e}")
        
        # 이커머스 시크릿만 삭제
        secrets_client.delete_secret(
            SecretId="ecommerce_customer_support_agent",
            ForceDeleteWithoutRecovery=True
        )
        print("  ✅ 이커머스 Secrets Manager 시크릿 삭제 완료")
        
    except secrets_client.exceptions.ResourceNotFoundException:
        print("  ⚠️ 이커머스 시크릿을 찾을 수 없습니다 (이미 삭제되었거나 생성되지 않음)")
    except Exception as e:
        print(f"  ⚠️ 이커머스 시크릿 정리 중 오류: {e}")
    
    # 이커머스 프로젝트의 IAM 역할만 삭제 (명시적 이름 패턴)
    try:
        print("  🗑️ 이커머스 IAM 역할 삭제 중...")
        ecommerce_roles_to_delete = [
            "EcommerceAgentCoreRuntimeExecutionRole",
            "EcommerceCustomerSupportRole",
            "EcommerceAgentCoreRole"
        ]
        
        for role_name in ecommerce_roles_to_delete:
            try:
                # 역할이 존재하는지 확인
                iam_client.get_role(RoleName=role_name)
                
                # 연결된 정책들 먼저 분리 (이커머스 역할에서만)
                attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
                for policy in attached_policies['AttachedPolicies']:
                    iam_client.detach_role_policy(
                        RoleName=role_name,
                        PolicyArn=policy['PolicyArn']
                    )
                
                # 인라인 정책들 삭제 (이커머스 역할에서만)
                inline_policies = iam_client.list_role_policies(RoleName=role_name)
                for policy_name in inline_policies['PolicyNames']:
                    iam_client.delete_role_policy(
                        RoleName=role_name,
                        PolicyName=policy_name
                    )
                
                # 이커머스 역할만 삭제
                iam_client.delete_role(RoleName=role_name)
                print(f"    ✅ 이커머스 역할 삭제 완료: {role_name}")
                
            except iam_client.exceptions.NoSuchEntityException:
                print(f"    ⚠️ 이커머스 역할을 찾을 수 없습니다: {role_name}")
            except Exception as e:
                print(f"    ❌ 이커머스 역할 삭제 실패 {role_name}: {e}")
                
    except Exception as e:
        print(f"  ❌ 이커머스 IAM 역할 정리 중 오류: {e}")
    
    # 이커머스 프로젝트의 SSM 파라미터들만 삭제
    try:
        print("  🗑️ 이커머스 SSM 파라미터들 삭제 중...")
        ecommerce_parameters_to_delete = [
            "/app/ecommerce/agentcore/memory_id",
            "/app/ecommerce/agentcore/runtime_arn",
            "/app/ecommerce/agentcore/gateway_id"
        ]
        
        for param_name in ecommerce_parameters_to_delete:
            try:
                ssm_client.delete_parameter(Name=param_name)
                print(f"    ✅ 이커머스 SSM 파라미터 삭제: {param_name}")
            except ssm_client.exceptions.ParameterNotFound:
                print(f"    ⚠️ 이커머스 SSM 파라미터를 찾을 수 없습니다: {param_name}")
            except Exception as e:
                print(f"    ❌ 이커머스 SSM 파라미터 삭제 실패 {param_name}: {e}")
                
    except Exception as e:
        print(f"  ❌ 이커머스 SSM 파라미터 정리 중 오류: {e}")
        
except Exception as e:
    print(f"❌ 이커머스 보안 리소스 정리 중 전체 오류: {e}")

print("✅ 이커머스 보안 리소스 정리 완료")

## Step 6: 이커머스 관측성 리소스 정리

이커머스 프로젝트 관련 CloudWatch 로그 그룹만 제거합니다.

In [ ]:
print("📊 이커머스 관측성 리소스 정리 시작...")

try:
    # 이커머스 프로젝트의 CloudWatch 로그 그룹만 삭제
    print("  🗑️ 이커머스 CloudWatch 로그 그룹 삭제 중...")
    ecommerce_log_groups_to_delete = [
        "/aws/lambda/ecommerce-return-eligibility",
        "/aws/lambda/ecommerce-web-search",
        "/aws/lambda/ecommerce-customer-support"
    ]
    
    # 이커머스 관련 로그 그룹들을 동적으로 찾기
    try:
        paginator = logs_client.get_paginator('describe_log_groups')
        for page in paginator.paginate():
            for log_group in page['logGroups']:
                log_group_name = log_group['logGroupName']
                # 이커머스 관련 로그 그룹만 식별
                if ('ecommerce' in log_group_name.lower() or 
                    'k-style' in log_group_name.lower() or
                    log_group_name in ecommerce_log_groups_to_delete):
                    ecommerce_log_groups_to_delete.append(log_group_name)
    except Exception as e:
        print(f"    ⚠️ 이커머스 로그 그룹 자동 탐지 실패: {e}")
    
    # 중복 제거
    ecommerce_log_groups_to_delete = list(set(ecommerce_log_groups_to_delete))
    
    for log_group in ecommerce_log_groups_to_delete:
        try:
            logs_client.delete_log_group(logGroupName=log_group)
            print(f"    ✅ 이커머스 로그 그룹 삭제: {log_group}")
        except logs_client.exceptions.ResourceNotFoundException:
            print(f"    ⚠️ 이커머스 로그 그룹을 찾을 수 없습니다: {log_group}")
        except Exception as e:
            print(f"    ❌ 이커머스 로그 그룹 삭제 실패 {log_group}: {e}")
            
except Exception as e:
    print(f"❌ 이커머스 관측성 리소스 정리 중 오류: {e}")

print("✅ 이커머스 관측성 리소스 정리 완료")

## Step 7: 이커머스 로컬 파일 정리

이커머스 프로젝트의 로컬 임시 파일들만 제거합니다.

In [ ]:
print("📁 이커머스 로컬 파일 정리 시작...")

import shutil

try:
    # 이커머스 프로젝트의 임시 파일들만 정리
    print("  🗑️ 이커머스 프로젝트 임시 파일들 삭제 중...")
    
    # 가상환경 외부의 __pycache__ 디렉토리들만 삭제
    import os
    for root, dirs, files in os.walk("."):
        if "__pycache__" in dirs:
            pycache_path = os.path.join(root, "__pycache__")
            # 가상환경 내부는 제외하고 이커머스 프로젝트 관련만
            if (".venv" not in pycache_path and 
                "site-packages" not in pycache_path and
                ("src" in pycache_path or "src/helpers" in pycache_path)):
                try:
                    shutil.rmtree(pycache_path)
                    print(f"    ✅ 이커머스 __pycache__ 삭제: {pycache_path}")
                except Exception as e:
                    print(f"    ❌ 이커머스 __pycache__ 삭제 실패 {pycache_path}: {e}")
    
    # 이커머스 프로젝트의 임시 디렉토리들만 삭제
    ecommerce_temp_directories = [
        "artifacts",
        "logs", 
        "temp",
        "ecommerce_artifacts"
    ]
    
    for directory in ecommerce_temp_directories:
        if os.path.exists(directory):
            try:
                shutil.rmtree(directory)
                print(f"    ✅ 이커머스 임시 디렉토리 삭제: {directory}")
            except Exception as e:
                print(f"    ❌ 이커머스 임시 디렉토리 삭제 실패 {directory}: {e}")
    
    print("  ✅ 이커머스 로컬 파일 정리 완료")
    
except Exception as e:
    print(f"❌ 이커머스 로컬 파일 정리 중 오류: {e}")

print("✅ 이커머스 로컬 파일 정리 완료")

## Step 8: 정리 완료 확인

이커머스 프로젝트 리소스들이 완전히 정리되었는지 확인합니다.

In [ ]:
print("🔍 이커머스 리소스 정리 상태 확인...")

verification_results = []

# SSM 파라미터 확인
try:
    ecommerce_params = [
        "/app/ecommerce/agentcore/memory_id",
        "/app/ecommerce/agentcore/runtime_arn", 
        "/app/ecommerce/agentcore/gateway_id"
    ]
    
    remaining_params = []
    for param in ecommerce_params:
        try:
            ssm_client.get_parameter(Name=param)
            remaining_params.append(param)
        except ssm_client.exceptions.ParameterNotFound:
            pass  # 정상적으로 삭제됨
    
    if remaining_params:
        verification_results.append(f"⚠️ 남은 이커머스 SSM 파라미터: {remaining_params}")
    else:
        verification_results.append("✅ 모든 이커머스 SSM 파라미터 정리 완료")
        
except Exception as e:
    verification_results.append(f"❌ SSM 파라미터 확인 실패: {e}")

# Secrets Manager 확인
try:
    secrets_client.get_secret_value(SecretId="ecommerce_customer_support_agent")
    verification_results.append("⚠️ 이커머스 시크릿이 아직 남아있습니다")
except secrets_client.exceptions.ResourceNotFoundException:
    verification_results.append("✅ 이커머스 시크릿 정리 완료")
except Exception as e:
    verification_results.append(f"❌ 시크릿 확인 실패: {e}")

# CloudFormation 스택 확인
try:
    ecommerce_stacks = ["ecommerce-customer-support-infrastructure", "ecommerce-customer-support-cognito"]
    remaining_stacks = []
    
    for stack_name in ecommerce_stacks:
        try:
            cloudformation_client.describe_stacks(StackName=stack_name)
            remaining_stacks.append(stack_name)
        except cloudformation_client.exceptions.ClientError:
            pass  # 정상적으로 삭제됨
    
    if remaining_stacks:
        verification_results.append(f"⚠️ 남은 이커머스 CloudFormation 스택: {remaining_stacks}")
    else:
        verification_results.append("✅ 모든 이커머스 CloudFormation 스택 정리 완료")
        
except Exception as e:
    verification_results.append(f"❌ CloudFormation 스택 확인 실패: {e}")

# 결과 출력
print("\n📋 정리 확인 결과:")
for result in verification_results:
    print(f"  {result}")

print("\n✅ 이커머스 리소스 정리 상태 확인 완료")

## 🎉 K-Style 이커머스 정리 완료!

이커머스 프로젝트에서 생성한 모든 AWS 리소스가 안전하게 정리되었습니다.

In [ ]:
print("\n" + "=" * 70)
print("🧹 K-STYLE 이커머스 에이전트 정리 완료! 🧹")
print("=" * 70)
print()
print("📋 정리된 이커머스 리소스들:")
print("  🧠 메모리: 이커머스 AgentCore Memory 리소스 및 데이터")
print("  🚀 런타임: 이커머스 Agent 런타임 및 ECR 리포지토리")
print("  ⚙️ 게이트웨이: 이커머스 AgentCore Gateway 및 Lambda 함수들")
print("  🛡️ 보안: 이커머스 IAM 역할, Cognito 사용자 풀, SSM 파라미터")
print("  📊 관측성: 이커머스 CloudWatch 로그 그룹 및 스트림")
print("  📁 파일: 이커머스 로컬 설정 및 임시 파일들")
print()
print("🔒 안전성 보장:")
print("  ✅ 다른 프로젝트의 리소스는 보호됨")
print("  ✅ 이커머스 프로젝트 리소스만 정확히 식별하여 삭제")
print("  ✅ 명시적 이름 패턴으로 안전한 삭제 보장")
print()
print("✨ AWS 계정에서 이커머스 프로젝트가 깔끔하게 정리되었습니다!")
print("🛍️ K-Style 이커머스 고객 지원 에이전트 워크숍을 완료해주셔서 감사합니다!")
print()
print("🎯 다음 단계:")
print("  • 새로운 에이전트 실험을 위해 Lab 1부터 다시 시작")
print("  • 다른 유스케이스에 AgentCore 적용 고려")
print("  • 프로덕션 환경에서의 확장 방안 검토")
print("  • 다른 AgentCore 튜토리얼과의 공존 가능")
print("\n" + "=" * 70)